In [1]:
from IPython.display import clear_output
!nvidia-smi

Fri Mar  4 17:54:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    35W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install tensorboardx
!pip install transformers
!pip install simpletransformers
clear_output()

In [3]:
import random
import os
from urllib import request
import pandas as pd
from sklearn.model_selection import StratifiedKFold,train_test_split,KFold
from collections import Counter,defaultdict
from sklearn.preprocessing import LabelEncoder,MultiLabelBinarizer
from sklearn.metrics import classification_report
import numpy as np

import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader
import seaborn as sns
from textblob import TextBlob, Word
import logging
import transformers

In [4]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/NLP/CW'
# %mkdir NLP_CW
# %cd NLP_CW
# %mkdir ref
# %mkdir res
# %mkdir models
# %mkdir outputs


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/NLP/CW


In [5]:
# prepare logger
logging.basicConfig(level=logging.INFO)

transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# check gpu
cuda_available = torch.cuda.is_available()

print('Cuda available? ',cuda_available)

if cuda_available:
  import tensorflow as tf
  # Get the GPU device name.
  device_name = tf.test.gpu_device_name()
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  # The device name should look like the following:
  if device_name == '/device:GPU:0':
      print('Found GPU at: {}'.format(device_name))
  else:
      raise SystemError('GPU device not found')

Cuda available?  True
Found GPU at: /device:GPU:0


2022-03-04 17:55:15.489596: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.


# Fetch Don't Patronize Me! data

In [6]:
module_url = f"https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py"
module_name = module_url.split('/')[-1]
print(f'Fetching {module_url}')
#with open("file_1.txt") as f1, open("file_2.txt") as f2
with request.urlopen(module_url) as f, open(module_name,'w') as outf:
  a = f.read()
  outf.write(a.decode('utf-8'))

Fetching https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/dont_patronize_me.py


In [7]:
# helper function to save predictions to an output file
def labels2file(p, outf_path):
	with open(outf_path,'w') as outf:
		for pi in p:
			outf.write(','.join([str(k) for k in pi])+'\n')

In [63]:
!wget https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/dev_semeval_parids-labels.csv
!wget https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/practice%20splits/train_semeval_parids-labels.csv
!wget https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/evaluation.py
!wget https://raw.githubusercontent.com/Perez-AlmendrosC/dontpatronizeme/master/semeval-2022/TEST/task4_test.tsv
!wget https://raw.githubusercontent.com/LordLean/SemEval-2022-Task-4/main/dontpatronizeme_v1.4/augmented.csv
!wget https://raw.githubusercontent.com/LordLean/SemEval-2022-Task-4/main/dontpatronizeme_v1.4/dontpatronizeme_pcl.tsv

--2022-03-04 16:26:17--  https://raw.githubusercontent.com/LordLean/SemEval-2022-Task-4/main/dontpatronizeme_v1.4/augmented.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 470940 (460K) [text/plain]
Saving to: ‘augmented.csv’

augmented.csv       100%[===================>] 459.90K  --.-KB/s    in 0.05s   

2022-03-04 16:26:17 (9.63 MB/s) - ‘augmented.csv’ saved [470940/470940]



In [8]:
aug_data_class1 = pd.read_csv('augmented.csv')
# print(aug_data)


def get_augmented(data):
    rows = []
    for idx in range(len(data)):  
      parid = data.par_id[idx]
      text = data.text[idx]
      label = data.label[idx]
      rows.append({
          'par_id':parid,
          'text':text,
          'label':label
      })
    return pd.DataFrame(rows)

aug_data_class1 = get_augmented(aug_data_class1)
# print(aug_data_class1)    

In [9]:
from dont_patronize_me import DontPatronizeMe
dpm = DontPatronizeMe('.', 'task4_test.tsv')
dpm.load_task1()

# Load paragraph IDs

In [10]:
trids = pd.read_csv('train_semeval_parids-labels.csv')
teids = pd.read_csv('dev_semeval_parids-labels.csv')

In [11]:
trids.par_id = trids.par_id.astype(str)
teids.par_id = teids.par_id.astype(str)



# Rebuild training set (Task 1)

In [12]:
rows = [] # will contain par_id, label and text
for idx in range(len(trids)):  
  parid = trids.par_id[idx]
  #print(parid)
  # select row from original dataset to retrieve `text` and binary label
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })
  

In [13]:
trdf1 = pd.DataFrame(rows)
print(trdf1)

     par_id                                               text  label
0      4341  The scheme saw an estimated 150,000 children f...      1
1      4136  Durban 's homeless communities reconciliation ...      1
2     10352  The next immediate problem that cropped up was...      1
3      8279  Far more important than the implications for t...      1
4      1164  To strengthen child-sensitive social protectio...      1
...     ...                                                ...    ...
8370   8380  Rescue teams search for survivors on the rubbl...      0
8371   8381  The launch of ' Happy Birthday ' took place la...      0
8372   8382  The unrest has left at least 20,000 people dea...      0
8373   8383  You have to see it from my perspective . I may...      0
8374   8384  Yet there was one occasion when we went to the...      0

[8375 rows x 3 columns]


# Rebuild test set (Task 1)

In [14]:
rows = [] # will contain par_id, label and text
for idx in range(len(teids)):  
  parid = teids.par_id[idx]
  #print(parid)
  # select row from original dataset
  text = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].text.values[0]
  label = dpm.train_task1_df.loc[dpm.train_task1_df.par_id == parid].label.values[0]
  rows.append({
      'par_id':parid,
      'text':text,
      'label':label
  })

tedf1 = pd.DataFrame(rows)
  

# Training set downsampling

In [15]:
augdf = pd.concat([trdf1, aug_data_class1])
augdf.reset_index(inplace=True, drop=True)
pcldf = augdf[augdf.label==1]
npos = len(pcldf)

augdf = pd.concat([pcldf,augdf[augdf.label==0][:npos*1]])

In [16]:
# downsample negative instances
pcldf = trdf1[trdf1.label==1]
npos = len(pcldf)

training_set1 = pd.concat([pcldf,trdf1[trdf1.label==0][:npos*2]])

In [21]:
# training_set1

,par_id,text,label
0,4341,"The scheme saw an estimated 150,000 children f...",1
1,4136,Durban 's homeless communities reconciliation ...,1
2,10352,The next immediate problem that cropped up was...,1
3,8279,Far more important than the implications for t...,1
4,1164,To strengthen child-sensitive social protectio...,1
...,...,...,...
2377,1775,Last but not the least element of culpability ...,0
2378,1776,"Then , taking the art of counter-intuitive non...",0
2379,1777,Kagunga village was reported to lack necessary...,0
2380,1778,"""After her parents high-profile divorce after ...",0


In [17]:
class Semeval(torch.utils.data.Dataset):

    def __init__(self, tokenizer, input_set):

        self.tokenizer = tokenizer
        self.texts = input_set['text']
        self.labels = input_set['label']
        
    def collate_fn(self, batch):

        texts = []
        labels = []

        for b in batch:
            texts.append(b['text'])
            labels.append(b['label'])

        #The maximum sequence size for ELECTRA is 512 but here the tokenizer truncate sentences longer than 128 tokens.  
        # We also pad shorter sentences to a length of 128 tokens
        encodings = self.tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=256)
        # labels = {}
        encodings['label'] =  torch.tensor(labels)
        
        return encodings
    
    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
       
        item = {'text': self.texts[idx],
                'label': self.labels[idx]}
        return item

# ELECTRA-small

In [18]:
from transformers import ElectraModel, ElectraConfig, ElectraPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers import ElectraTokenizer

In [19]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
tokenizer

PreTrainedTokenizer(name_or_path='google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [25]:
# # Analysing baseline Electra
# model = ElectraModel.from_pretrained("google/electra-small-discriminator")
# print(f"Model size: {model.num_parameters()}") #13.5 M
# model

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraModel: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight']
- This IS expected if you are initializing ElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Model size: 13483008


In [20]:
class Electra_Patronizing(ElectraPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # ELECTRA Model
        self.electra = ElectraModel(config)
        self.projection = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(config.hidden_size, 1),
            # torch.nn.ReLU(),
            # torch.nn.Dropout(0.2),
            # torch.nn.Linear(32, 1),
            torch.nn.Sigmoid()
        )
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.electra(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # print(outputs[0].size())
        outputs = torch.mean(outputs[0], dim=1)
        # print(outputs.size())

        logits = self.projection(outputs)
        
        return logits


In [21]:
class Trainer_patronizing(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs).float()
        

        loss_task = torch.nn.BCELoss()
        labels = labels['label'].float()
        # loss = loss_task(outputs.view(-1, 2), labels.view(-1))
        # print(outputs)
        loss = loss_task(outputs.squeeze(), labels)
        
        return loss

In [65]:
def main_train(train_dataset, model):
    
    training_args = TrainingArguments(
        output_dir='./outputs',
        learning_rate = 0.00011,
        logging_steps= 100,
        per_device_train_batch_size=32,
        num_train_epochs = 4,
    )
    trainer = Trainer_patronizing(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/electra_finetuned/')

In [66]:
#train
train_dataset = Semeval(tokenizer, training_set1)
# aug_df = pd.concat([trdf1, augdf])
# aug_df.reset_index(inplace=True, drop=True)
# train_dataset = Semeval(tokenizer, aug_df)
model = Electra_Patronizing.from_pretrained("google/electra-small-discriminator")
main_train(train_dataset, model)

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache":

Step,Training Loss
100,0.524800
200,0.362100
300,0.242400




Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to ./models/electra_finetuned/
Configuration saved in ./models/electra_finetuned/config.json
Model weights saved in ./models/electra_finetuned/pytorch_model.bin


# Evaluation

In [51]:
def predict_electra(input, tokenizer, model): 
    model.eval() 
    encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    encodings = encodings.to(device=device)
    
    output = model(**encodings)
    preds = torch.round(output)

    return {'prediction':preds, 'confidence':output[0]}

In [52]:
from tqdm import tqdm

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader):
      # print(data)

      labels = {}
      labels['label'] = data['label']

      text_data = data['text']

      pred = predict_electra(text_data, tokenizer, model)

      preds.append(pred['prediction'][0].cpu().tolist())
      tot_labels.append(labels['label'][0].tolist())

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score
  report = classification_report(tot_labels, preds, labels=[0.0, 1.0], target_names=["Not Patro","Patro"], output_dict= True)

  return report

In [67]:
test_loader = Semeval(tokenizer, tedf1)
test_loader = DataLoader(test_loader)
report = evaluate(model, tokenizer, test_loader)

100%|██████████| 2094/2094 [00:32<00:00, 65.32it/s]


In [68]:
print(report)
print(f"\n\noverall accuracy: {report['accuracy']}")

print(f"\nRecall for non-patronizing class: {report['Not Patro']['recall']}")
print(f"Precision for non-patronizing class: {report['Not Patro']['precision']}")
print(f"F1-score for non-patronizing class: {report['Not Patro']['f1-score']}")

print(f"\nRecall for patronizing class: {report['Patro']['recall']}")
print(f"Precision for patronizing class: {report['Patro']['precision']}")
print(f"F1-score for patronizing class: {report['Patro']['f1-score']}")

{'Not Patro': {'precision': 0.9685497961560862, 'recall': 0.8775725593667546, 'f1-score': 0.9208194905869324, 'support': 1895}, 'Patro': {'precision': 0.38461538461538464, 'recall': 0.7286432160804021, 'f1-score': 0.5034722222222222, 'support': 199}, 'accuracy': 0.8634192932187201, 'macro avg': {'precision': 0.6765825903857354, 'recall': 0.8031078877235783, 'f1-score': 0.7121458564045773, 'support': 2094}, 'weighted avg': {'precision': 0.9130565068071846, 'recall': 0.8634192932187201, 'f1-score': 0.8811575486554246, 'support': 2094}}


overall accuracy: 0.8634192932187201

Recall for non-patronizing class: 0.8775725593667546
Precision for non-patronizing class: 0.9685497961560862
F1-score for non-patronizing class: 0.9208194905869324

Recall for patronizing class: 0.7286432160804021
Precision for patronizing class: 0.38461538461538464
F1-score for patronizing class: 0.5034722222222222


# RoBERTa large

In [36]:
from transformers import RobertaModel, RobertaConfig, RobertaPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer

# # Initializing a ELECTRA electra-base-uncased style configuration
# configuration = ElectraConfig()

# # Initializing a model from the electra-base-uncased style configuration
# model = ElectraModel(configuration)

# # Accessing the model configuration
# configuration = model.config

In [37]:
tokenizer = RobertaTokenizer.from_pretrained('roberta-large')
tokenizer

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/482 [00:00<?, ?B/s]

PreTrainedTokenizer(name_or_path='roberta-large', vocab_size=50265, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'sep_token': AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'cls_token': AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'mask_token': AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True)})

In [ ]:
# Analysing baseline Electra
# model = RobertaModel.from_pretrained('roberta-large')
# print(f"Model size: {model.num_parameters()}") #355 M
# model

In [38]:
class Roberta_Patronizing(RobertaPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # Roberta Model
        self.electra = RobertaModel(config)
        self.projection = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(config.hidden_size, 2),
            torch.nn.Softmax()
        )
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.electra(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        logits = self.projection(outputs[1])
        
        return logits


In [39]:
class Trainer_patronizing(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs).float()
        

        loss_weighted = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.5,1]).to(device))
        labels = labels['label']
        # print(outputs.size())
        # print(labels.size())
        # print(outputs)
        # print(labels)
        loss = loss_weighted(outputs.squeeze(), labels)
        
        return loss

In [40]:
def main_train(train_dataset, model):
    
    training_args = TrainingArguments(
        output_dir='./outputs',
        learning_rate = 1e-5,
        logging_steps= 100,
        per_device_train_batch_size=3,
        num_train_epochs = 10,
        weight_decay=1e-8,
    )
    trainer = Trainer_patronizing(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/roberta_finetuned/')

In [41]:
#train
train_dataset = Semeval(tokenizer, training_set1)
model = Roberta_Patronizing.from_pretrained('roberta-large')
main_train(train_dataset, model)

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-large were not used when initializing Roberta_Patronizing: ['roberta.encoder.layer.20.attention.output.dense.bias', 'roberta.encoder.layer.17.attention.output.LayerNorm.bias', 'roberta.encoder.layer.23.intermediate.dense.weight', 'roberta.encoder.layer.20.output.LayerNorm.weight', 'roberta.encoder.layer.15.intermediate.dense.weight', 'roberta.encoder.layer.3.attention.self.value.bias', 'roberta.encoder.layer.4.attention.self.value.weight', 'roberta.encoder.layer.20.output.LayerNorm.bias', 'roberta.encoder.layer.20.attention.output.LayerNorm.bias', 'roberta.encoder.layer.12.output.dense.bias', 'roberta.encoder.layer.5.output.dense.weight', 'roberta.encoder.layer.4.output.LayerNorm.weight', 'roberta.encoder.layer.14.output.LayerNorm.bias', 'roberta.encoder.layer.14.attention.output.LayerNorm.bias', 'roberta.encoder.layer.6.output.LayerNorm.weight', 'roberta.encoder.layer.7.attention.output.LayerNorm.weight', 'roberta.encoder.layer.5.attenti

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


Step,Training Loss
100,0.734600
200,0.677200
300,0.693900
400,0.678300
500,0.726200
600,0.709700
700,0.702100
800,0.696900
900,0.695400
1000,0.700500


Saving model checkpoint to ./outputs/checkpoint-500
Configuration saved in ./outputs/checkpoint-500/config.json
Model weights saved in ./outputs/checkpoint-500/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Saving model checkpoint to ./outputs/checkpoint-1000
Configuration saved in ./outputs/checkpoint-1000/config.json
Model weights saved in ./outputs/checkpoint-1000/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Saving model checkpoint to ./outputs/checkpoint-1500
Configuration saved in ./outputs/checkpoint-1500/config.json
Model weights saved in ./outputs/checkpoint-1500/pytorch_model.bin
/usr/local/lib/python3

In [46]:
def predict_roberta(input, tokenizer, model): 
    model.eval() 
    encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    encodings = encodings.to(device=device)
    
    output = model(**encodings)
    # print(output.size())
    preds = torch.argmax(output, 1)

    return {'prediction':preds, 'confidence':output[preds]}

In [43]:
from tqdm import tqdm

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader):
      # print(data)

      labels = {}
      labels['label'] = data['label']

      text_data = data['text']

      pred = predict_roberta(text_data, tokenizer, model)

      preds.append(pred['prediction'][0].cpu().tolist())
      # print(len(preds[0]))
      tot_labels.append(labels['label'][0].tolist())
      # print(tot_labels)

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score

  report = classification_report(tot_labels, preds, labels=[0.0, 1.0], target_names=["Not Patro","Patro"], output_dict= True)

  return report

In [47]:
test_loader = Semeval(tokenizer, tedf1)
test_loader = DataLoader(test_loader)
report = evaluate(model, tokenizer, test_loader)

print(report)

print(report['accuracy'])
print(report["Not Patro"]['f1-score'])
print(report["Patro"]['f1-score'])

  0%|          | 0/2094 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
100%|██████████| 2094/2094 [00:59<00:00, 35.37it/s]

{'Not Patro': {'precision': 0.9049665711556829, 'recall': 1.0, 'f1-score': 0.9501128102281273, 'support': 1895}, 'Patro': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 199}, 'accuracy': 0.9049665711556829, 'macro avg': {'precision': 0.45248328557784145, 'recall': 0.5, 'f1-score': 0.47505640511406366, 'support': 2094}, 'weighted avg': {'precision': 0.8189644949092737, 'recall': 0.9049665711556829, 'f1-score': 0.8598203320832384, 'support': 2094}}
0.9049665711556829
0.9501128102281273
0.0



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# ELECTRA two-class

In [23]:
from transformers import ElectraModel, ElectraConfig, ElectraPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers import ElectraTokenizer

# # Initializing a ELECTRA electra-base-uncased style configuration
# configuration = ElectraConfig()

# # Initializing a model from the electra-base-uncased style configuration
# model = ElectraModel(configuration)

# # Accessing the model configuration
# configuration = model.config

In [61]:
tokenizer = ElectraTokenizer.from_pretrained("google/electra-small-discriminator")
tokenizer

loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/ece45ade3e01224cf31fed8e183b306d17b84e8abd415363474cfe72274f7814.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/google/electra-small-discriminator/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/8b3aea9f7242b3d19268df5b1bfed8f66e08671a72ac0809ada08e5ef1adc592.19eda9a6da5fb0e52a45200c95876729561dde16a69b9116953af6edca1d1e92
loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3af

PreTrainedTokenizer(name_or_path='google/electra-small-discriminator', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [25]:
class Electra_Patronizing(ElectraPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # ELECTRA Model
        self.electra = ElectraModel(config)
        self.projection = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(config.hidden_size, 2),
            # torch.nn.ReLU(),
            # torch.nn.Dropout(0.2),
            # torch.nn.Linear(32, 1),
            torch.nn.Softmax()
        )
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.electra(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        # print(outputs[0].size())
        outputs = torch.mean(outputs[0], dim=1)
        # print(outputs.size())

        logits = self.projection(outputs)
        
        return logits


In [26]:
class Trainer_patronizing(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs).float()
        

        loss_weighted = torch.nn.CrossEntropyLoss(weight=torch.tensor([0.5,1]).to(device))
        labels = labels['label']
        loss = loss_weighted(outputs.squeeze(), labels)
        
        return loss

In [27]:
def main_train(train_dataset, model):
    
    training_args = TrainingArguments(
        output_dir='./outputs',
        learning_rate = 0.0001,
        logging_steps= 100,
        per_device_train_batch_size=32,
        num_train_epochs = 5,
    )
    trainer = Trainer_patronizing(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/electra_finetuned/')

In [62]:
#train
train_dataset = Semeval(tokenizer, training_set1)
model = Electra_Patronizing.from_pretrained("google/electra-small-discriminator")
main_train(train_dataset, model)

loading configuration file https://huggingface.co/google/electra-small-discriminator/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ca13c16218c6780ec76753d3afa19fcb7cc759e3f63ee87e441562d374762b3d.3dd1921e571dfa18c0bdaa17b9b38f111097812281989b1cb22263738e66ef73
Model config ElectraConfig {
  "architectures": [
    "ElectraForPreTraining"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache":

RuntimeError: ignored

In [59]:
def predict_electra(input, tokenizer, model): 
    model.eval() 
    encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    # print(device)
    print(encodings)
    encodings = encodings[0].to(device=device)
    
    output = model(**encodings)
    preds = torch.argmax(output, 1)

    return {'prediction':preds, 'confidence':output[preds]}

In [41]:
from tqdm import tqdm

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader):
      # print(data)

      labels = {}
      labels['label'] = data['label']

      text_data = data['text']

      pred = predict_electra(text_data, tokenizer, model)

      preds.append(pred['prediction'][0].cpu().tolist())
      # print(len(preds[0]))
      tot_labels.append(labels['label'][0].tolist())
      # print(tot_labels)

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score

  report = classification_report(tot_labels, preds, labels=[0.0, 1.0], target_names=["Not Patro","Patro"], output_dict= True)

  return report

In [60]:
test_loader = Semeval(tokenizer, tedf1)
test_loader = DataLoader(test_loader)
report = evaluate(model, tokenizer, test_loader)

print(report)

print(report['accuracy'])
print(report["Not Patro"]['f1-score'])
print(report["Patro"]['f1-score'])

  0%|          | 0/2094 [00:00<?, ?it/s]

{'input_ids': tensor([[  101,  2057,  2036,  2113,  2008,  2027,  2064,  5770,  2011,  4909,
         17041,  2013,  2619,  2040,  2064,  2393,  2068,  3305,  2008,  2037,
          5346,  2024,  3671,  1998,  2008,  2037,  3663,  2003,  2025, 20625,
          1025,  2619,  2040,  2064,  2393,  2068,  2404,  2037,  3663,  1999,
          7339,  1998,  2393,  2068, 10639,  2007,  2500,  2040,  2071,  3073,
          2490,  1025,  2619,  3716,  3085,  2055,  4219,  2027,  2064,  3229,
          1025,  2619,  2040,  2064,  2393,  2068,  2933,  2005,  2037,  3791,
          1998,  1996,  3791,  1997,  2037,  2775,  2011,  4975,  2593,  1037,
         28586,  2933,  2030,  2019,  9886,  2933,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0

KeyError: ignored

# RoBERTa Baseline

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs, MultiLabelClassificationModel, MultiLabelClassificationArgs
task1_model_args = ClassificationArgs(num_train_epochs=1, 
                                      no_save=True, 
                                      no_cache=True, 
                                      overwrite_output_dir=True)
task1_model = ClassificationModel("roberta", 
                                  'roberta-base', 
                                  args = task1_model_args, 
                                  num_labels=2, 
                                  use_cuda=cuda_available)
# train model
task1_model.train_model(training_set1[['text', 'label']])
# run predictions
preds_task1, _ = task1_model.predict(tedf1.text.tolist())

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/298 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of roberta model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/2094 [00:00<?, ?it/s]

  0%|          | 0/262 [00:00<?, ?it/s]

In [ ]:
Counter(preds_task1)

Counter({0: 1675, 1: 419})

In [ ]:
labels2file([[k] for k in preds_task1], 'task1.txt')

In [ ]:
import os

dpm.load_test()
labels2file([[k] for k in preds_task1], 'res/task1.txt')
labels2file(tedf1.label.apply(lambda x:[x]).tolist(), 'ref/task1.txt')
!python evaluation.py . .
!cat scores.txt

task1_precision:0.36754176610978523
task1_recall:0.7738693467336684
task1_f1:0.4983818770226537


In [ ]:
res = [[random.choice([0,1]) for k in range(7)] for k in range(0,len(teids))]

In [ ]:
labels2file(res, 'task2.txt')

# distilBERT

In [ ]:
from transformers import DistilBertModel, DistilBertConfig, DistilBertPreTrainedModel
from transformers import Trainer, TrainingArguments
from transformers import DistilBertTokenizer

# # Initializing a ELECTRA electra-base-uncased style configuration
# configuration = ElectraConfig()

# # Initializing a model from the electra-base-uncased style configuration
# model = ElectraModel(configuration)

# # Accessing the model configuration
# configuration = model.config

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
tokenizer

In [ ]:
# Analysing baseline Electra
# model = DistilBertModel.from_pretrained('distilbert-base-uncased')
# print(f"Model size: {model.num_parameters()}") #355 M
# model

In [ ]:
class DistilBert_Patronizing(DistilBertPreTrainedModel):

    def __init__(self, config):
        super().__init__(config)

        # DistilBert Model
        self.distilbert = DistilBertModel(config)
        self.projection = torch.nn.Sequential(
            torch.nn.Dropout(0.2),
            torch.nn.Linear(config.hidden_size, 2),
            torch.nn.Softmax()
        )
        
        self.init_weights()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None):
 
        outputs = self.distilbert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        logits = self.projection(outputs[1])
        
        return logits


In [ ]:
class Trainer_patronizing(Trainer):
    def compute_loss(self, model, inputs):
        labels = {}
        labels['label'] = inputs.pop('label')

        outputs = model(**inputs).float()
        

        loss_task = torch.nn.BCELoss()
        labels = labels['label'].float()
        loss = loss_task(outputs.squeeze(), labels)
        
        return loss

In [ ]:
def main_train(train_dataset, model):
    
    training_args = TrainingArguments(
        output_dir='./outputs',
        learning_rate = 0.001,
        logging_steps= 100,
        per_device_train_batch_size=32,
        num_train_epochs = 10,
        # weight_decay=1e-8,
    )
    trainer = Trainer_patronizing(
        model=model,                         
        args=training_args,                 
        train_dataset=train_dataset,                   
        data_collator=train_dataset.collate_fn
    )
    trainer.train()

    trainer.save_model('./models/distilbert_finetuned/')

In [ ]:
#train
train_dataset = Semeval(tokenizer, training_set1)
model = DistilBert_Patronizing.from_pretrained('distilbert-base-uncased')
main_train(train_dataset, model)

In [ ]:
def predict_DistilBert(input, tokenizer, model): 
    model.eval() 
    encodings = tokenizer(input, return_tensors='pt', padding=True, truncation=True, max_length=128)
    encodings = encodings.to(device=device)
    
    output = model(**encodings)
    print(output.size())
    preds = torch.max(output, 1)
    print(preds[1])

    return {'prediction':preds[0], 'confidence':preds[1]}

In [ ]:
from tqdm import tqdm

def evaluate(model, tokenizer, data_loader):

  total_count = 0
  correct_count = 0 

  preds = []
  tot_labels = []

  with torch.no_grad():
    for data in tqdm(data_loader):
      # print(data)

      labels = {}
      labels['label'] = data['label']

      text_data = data['text']

      pred = predict_DistilBert(text_data, tokenizer, model)

      preds.append(pred['prediction'][0].cpu().tolist())
      # print(len(preds[0]))
      tot_labels.append(labels['label'][0].tolist())
      # print(tot_labels)

  # with the saved predictions and labels we can compute accuracy, precision, recall and f1-score

  report = classification_report(tot_labels, preds, labels=[0.0, 1.0], target_names=["Not Patro","Patro"], output_dict= True)

  return report

In [ ]:
test_loader = Semeval(tokenizer, tedf1)
test_loader = DataLoader(test_loader)
report = evaluate(model, tokenizer, test_loader)

print(report)

print(report['accuracy'])
print(report["Not Patro"]['f1-score'])
print(report["Patro"]['f1-score'])

# Random baseline



In [ ]:
res = [[random.choice([0,1])] for k in range(len(teids))]

In [ ]:
labels2file(res, 'task1.txt')

# Task 2

## Prepare submission

We first check the contents of the two files we have created, and compress them into a codalab-compatible `submission.zip` file.

In [ ]:
!cat task1.txt | head -n 10

0
0
0
1
1
0
1
1
0
1


In [ ]:
!cat task2.txt | head -n 10

1,1,1,1,1,1,1
0,0,0,0,0,1,0
0,1,1,0,0,0,1
1,0,0,0,1,1,0
0,0,0,1,1,0,0
1,0,0,1,1,1,0
0,0,1,0,0,1,0
1,1,1,1,0,1,1
1,1,0,0,0,0,0
0,1,0,0,0,0,1


In [ ]:
# the left pane should now show a file called submission.zip, which you can submit to Codalab
!zip submission.zip task1.txt task2.txt

updating: task1.txt (deflated 89%)
updating: task2.txt (deflated 87%)
